In [1]:
import glob
import numpy as np
from scipy.io import wavfile
import tensorflow as tf

In [2]:
TRAINING_SET_SIZE = 100
TEST_SET_SIZE = 824
FRAME_RATE = 48000

WINDOW_LENGTH_MILISECONDS = 10
WINDOW_SHIFTS_MILISECONDS = 10

TRAIN_SET_NOISY = '../data/noisy_trainset_28spk_wav/*.wav'
TRAIN_SET_CLEAN = '../data/clean_trainset_28spk_wav/*.wav'

TEST_SET_NOISY  = '../data/noisy_testset_wav/*.wav'
TEST_SET_CLEAN  = '../data/clean_testset_wav/*.wav'

In [3]:
sequence_length = int(FRAME_RATE*WINDOW_LENGTH_MILISECONDS/1000)
sequence_shift  = int(FRAME_RATE*WINDOW_SHIFTS_MILISECONDS/1000)
print(sequence_length, sequence_shift)

480 480


In [4]:
def min_max_scaling(Z):
    return (Z - np.min(Z))/np.ptp(Z)

def window_generator(path_to_noisy_files: np.array, path_to_clean_files: np.array, shift, length):

    noisy_filenames = glob.glob(path_to_noisy_files)
    noisy_filenames = glob.glob(path_to_clean_files)

    noisy_data = [wavfile.read(file)[1] for file in noisy_filenames]
    clean_data = [wavfile.read(file)[1] for file in noisy_filenames]

    while True:
        for i in range(len(noisy_data)):
            X_i = []
            Y_i = []
            for j in range((len(noisy_data[i]) - length)//shift + 1):
                X_i.append(min_max_scaling(noisy_data[i][j*shift: j*shift + length]))
                Y_i.append(min_max_scaling(clean_data[i][j*shift: j*shift + length]))
            yield np.array(X_i), np.array(Y_i)

train_generator = window_generator(TRAIN_SET_NOISY, TRAIN_SET_NOISY, shift=sequence_shift, length=sequence_length)
test_generator = window_generator(TEST_SET_NOISY, TEST_SET_CLEAN, shift=sequence_shift, length=sequence_length)

In [5]:
def RNN_model(sequence_length):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[None]),
        tf.keras.layers.SimpleRNN(sequence_length, return_sequences=True),
        tf.keras.layers.SimpleRNN(sequence_length),
        tf.keras.layers.Dense(1)])
    optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=optimizer, metrics=["mae"])
    return model

In [6]:
model = RNN_model(sequence_length)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              (None, None, 1)           0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 480)         231360    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 480)               461280    
_________________________________________________________________
dense (Dense)                (None, 1)                 481       
Total params: 693,121
Trainable params: 693,121
Non-trainable params: 0
_________________________________________________________________


In [9]:
hist = model.fit(train_generator,
                    validation_data=test_generator,
                    steps_per_epoch=TRAINING_SET_SIZE,
                    validation_steps=TEST_SET_SIZE,
                    epochs=10)

Epoch 1/10
100/100 [==============================] - 73s 729ms/step - loss: 0.0870 - mae: 0.3288 - val_loss: 0.1086 - val_mae: 0.3693
Epoch 2/10
100/100 [==============================] - 74s 741ms/step - loss: 0.0870 - mae: 0.3294 - val_loss: 0.1082 - val_mae: 0.3685
Epoch 3/10
100/100 [==============================] - 74s 741ms/step - loss: 0.0846 - mae: 0.3246 - val_loss: 0.1078 - val_mae: 0.3677
Epoch 4/10
100/100 [==============================] - 73s 732ms/step - loss: 0.0808 - mae: 0.3172 - val_loss: 0.1074 - val_mae: 0.3670
Epoch 5/10
100/100 [==============================] - 74s 741ms/step - loss: 0.0836 - mae: 0.3226 - val_loss: 0.1070 - val_mae: 0.3662
Epoch 6/10
100/100 [==============================] - 74s 736ms/step - loss: 0.0838 - mae: 0.3226 - val_loss: 0.1067 - val_mae: 0.3655
Epoch 7/10
100/100 [==============================] - ETA: 0s - loss: 0.0845 - mae: 0.3240

KeyboardInterrupt: 